In [ ]:
from model import TracableNN
from model_config import config
import torch
import pickle
import subprocess
import pandas as pd




In [ ]:
# Run the main process in main.py to execute the training loop and save the model's learnable parameters
subprocess.run(["python", "main.py"])

In [ ]:
# Initialize the model structure (without re-training)
# This is necessary to run the model in inference mode later
input_size = config['input_size']
hidden_size = config['hidden_size']
output_size = config['output_size']
num_epochs = config['num_epochs']
model = TracableNN(input_size, hidden_size, output_size, num_epochs)

# Load the trained model's state
model.load_state_dict(torch.load('outputs/trained_model.pth'))

# Switch the model to evaluation mode
model.eval()

with open('outputs/network_trace.pkl', 'rb') as f:
    network_trace = pickle.load(f)
model.network_trace = network_trace


# Check if the trace has data
if network_trace.trace:
    print(f"Trace data loaded with epochs: {list(network_trace.trace.keys())}")
else:
    print("Error: Trace data is empty or not loaded correctly.")

### Data Preprocessing 
We want to prepare our recorded network_trace schema data by converting it into a dataframe suitable for analysis and computation.
First, we define a function `create_final_classification_series` which converts final classifcation results into a list for target variable y. 
We then define `convert_nn_schema_to_df` which converts our network trace schema into a multi-indexed dataframe.

In [ ]:
# We want to get the data for our y variable, which is all the final classification results 
def create_final_classification_series(network_trace):
    # Convert the final classification results dictionary to a flattened list
    final_results = network_trace.final_classification_results

    # Initialize an empty list to hold all flattened classification results
    flattened_results = []

    # Iterate over the epochs and flatten the classification results
    for epoch, result in final_results.items():
        if isinstance(result, torch.Tensor):
            # Convert the tensor to a numpy array, flatten it, and extend the flattened_results list
            flattened_results.extend(result.cpu().numpy().flatten().tolist())
        else:
            flattened_results.extend(result)

    # Convert the flattened list to a pandas Series
    series = pd.Series(flattened_results, name="Class_Result")

    return series

# Example usage
classification_series = create_final_classification_series(network_trace)

# Example usage
classification_series = create_final_classification_series(network_trace)
display(classification_series)

In [ ]:
from helpers import get_model_level_integer

def convert_nn_schema_to_df(network_trace):
    # Create a list to hold the data for each neuron
    data = []
    # Create a list to hold the multi-level index (epoch, layer, neuron)
    index = []

    # Retrieve final classification results
    final_classification_results = network_trace.final_classification_results

    for epoch, layers in network_trace.trace.items():
        for layer_name, neurons in layers.items():
            layer_weights = network_trace.weights[epoch][layer_name]
            
            # Convert the weights tensor to a numpy array if it's a tensor
            if isinstance(layer_weights, torch.Tensor):
                layer_weights = layer_weights.numpy()
            
            num_input_connections = layer_weights.shape[1]  # This determines the number of input connections

            for neuron_id, neuron_data in neurons.items():
                neuron_id_int = get_model_level_integer(neuron_id) % layer_weights.shape[0]
                
                if neuron_id_int >= len(layer_weights):
                    print(f"Index {neuron_id_int} out of bounds for layer {layer_name} with size {len(layer_weights)}")
                    continue
                
                # Extract the weights for this specific neuron
                weight_for_neuron = layer_weights[neuron_id_int].tolist()
                
                # Pad the weights with None or a small number to ensure consistent column size
                weight_for_neuron.extend([0.00000] * (num_input_connections - len(weight_for_neuron)))
                
                # Append the weights to the data list
                data.append(weight_for_neuron)
                # Append the index (epoch, layer, neuron_id)
                index.append((epoch, layer_name, neuron_id))

    # Create the MultiIndex
    multi_index = pd.MultiIndex.from_tuples(index, names=["Epoch", "Layer", "Neuron"])

    # Create the DataFrame
    df = pd.DataFrame(data, index=multi_index)

    # Retrieve final classification results for each epoch and map to DataFrame
    classification_series = pd.Series(final_classification_results).map(lambda x: x.cpu().numpy().flatten()[0] if isinstance(x, torch.Tensor) else x)
    classification_series.name = "Final_Classification_Result"

    # Join the classification results to the main DataFrame
    df = df.join(classification_series, on="Epoch")

    # Return the DataFrame with NaNs filled with a small number
    return df.fillna(0.000000)
model_df = convert_nn_schema_to_df(network_trace)
display(model_df[0:31])


In [ ]:
# exampledf = model_df[0:31]
# from pathlib import Path  
# filepath = Path('outputs/data_slice.csv')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# exampledf.to_csv(filepath) 

### What to do about non-orthogonality
For a typical fully connected network, the shape of the weight tensor is `(output_features, input_features)`. Unless a network is a perfect cube (with all layers having the same number of output features and input features) a dataframe of a network schema will result in empty values for neurons in layers that have less input features than the layer with the maximum number of input features. 

So in the example of 
L_1 = {n_0, ..., n_19} with  shape: (10,20)
L_2 = {n_20, ... n_29}  with shape: (10, 10)
L_3 = {n_30} with shape: (1, 10)
... 
Layer 2 (L_2) will have 10 fewer values per neuron than its previous layer, as it will have 10 fewer input connections. 

This makes sense intuitively because a layer with 20 neurons will take 20 input connections but give 10 connections to every neuron in the next layer with 10 neurons. But if the next layer only has 10 neurons, it will only pass 10 connections to the next layer.

Something must thus be done about the holes in the data for all smaller layers  as many ML algorithms will not work with NaNs. 

Imputing 0 with for the NaNs probably makes the most sense since it effectively means that value is nothing

-----------------------------------------------------------------------------------------------------

### Column alignment problem for the neuron level

Since a dataframe is "right angled", the largest layer in the network will define an upper boundary for the features. This means only the layer with 20 input neurons will have a full table in the dataframe and all layers in the network will have empty values where the difference between its size and the max layer size would be.

 If we were to index the dataframe to the max_neurons, we would have sparse table that is difficult to work with.  

 So it is important to remember that columns 0-19 at the neuron level are  *not a universal point of reference*.
 We have to be careful how we calculate our tables so that we aren't mistakenly comparing values at different locations on the network but that have congruent columns on the dataframe. 

 Later, we will repackage our weights into tuples to keep track of their actual associations. 
 




------------------------------------------
### Analyzing Model Metadata 

#### Meta-model training 
This next phase involves taking the data from the model and modeling it. This is to help us select which neurons are the most associated with given outputs. We will then use that data to identify our model's proposed "structures". 

The idea is to use machine learning *reflectively* on the results of machine learning, to see if we can find meaningful patterns in the numbers it produces.

------------------------------------------

### Structure Identification

#### What is a structure? 
The central thesis of this experiment is that we can gain clarity about a model's inner workings by identifing what nodes in the network participate most significantly in contributing to certain outputs. 

If we can identify the "where" (by neural id and layer) and "when" (by epoch, or perhaps in future iterations, batch id) these neurons contribute to certain outputs, we can define a structure associated with that ouput.

This structure would be analogous to identifying what part of the brain is involved in producing language, or recognizing certain shapes, or perceiving sounds, etc. 

These structures are effectively the embodiment of the model's "concept" of the output. 

With the identification of structures, we can then label them, just as we label regions of the brain (Broca's area, the anterior cingulate gyrus, etc). And once we label something, we can begin to reference it, it has a chance to become something we can place programmatic hooks into. We could theoretically delete or change these structures programatically once we identify them. 


-----------------------------------------
### Methods of Analysis 
There is room here for any type of analysis that can be left to discretion of the researcher. At first glance it's not clear what approach would be most insightful. The idea is to experiment and find out.

Some options might include:

- Time-series analysis (by observing changes across neural populations or layers across epochs)
- Layer-wise analysis (by observing changes in state of a layer or interactions between layers)
- Aggregate neural analysis (identify pairwise connections, strong correlations between neurons or profile a single neuron's metrics )
- Markov Chains or Hidden Markov Models 
- Heatmaps, clustering, correlational analysis

The ultimate goal is to apply various machine learning methods experimentally to see if any meaningful patterns can be discovered in network metadata. 

Below will be a sampling of various methods that will be periodically added to and expanded. 

In [ ]:
import pandas as pd

def aggregate_layers_by_type(df):
    # Create a dictionary to store DataFrames for each layer type
    layer_dfs = {}

    # Get all unique layer types
    layer_types = df.index.get_level_values('Layer').unique()
    
    # Iterate over each layer type
    for layer in layer_types:
        layer_df = df.xs(layer, level='Layer')
        
        # Ensure that the DataFrame contains only numerical values (ints or floats)
        layer_df = layer_df.applymap(lambda x: x if isinstance(x, (int, float)) else 0)
        
        layer_dfs[layer] = layer_df

    return layer_dfs

def aggregate_neurons_across_layers(layer_dfs):
    # Concatenate all DataFrames in layer_dfs
    concatenated_df = pd.concat(layer_dfs.values())

    # Extract neuron types from the index
    concatenated_df.index = concatenated_df.index.map(lambda x: x[1])  
    
    # Ensure that the DataFrame contains only numerical values (ints or floats)
    concatenated_df = concatenated_df.applymap(lambda x: x if isinstance(x, (int, float)) else 0)
    
    # Group by neuron type and aggregate (e.g., mean, sum, etc.)
    aggregated_df = concatenated_df.groupby(level=0).mean()

    return aggregated_df


layer_dfs = aggregate_layers_by_type(model_df)
neuron_dfs = aggregate_neurons_across_layers(layer_dfs)

display(neuron_dfs)

L_input_df = layer_dfs['L_input']
L_hidden_1_df = layer_dfs['L_hidden_1']
L_output_df = layer_dfs['L_output']

def compute_epoch_average(df):
    # Group by Layer and Neuron to compute averages across all epochs
    average_df = df.groupby(level=["Layer", "Neuron"]).mean()

    # Create a new index for "Epoch_average" with the same Layer and Neuron combinations
    average_df = average_df.reset_index()
    average_df['Epoch'] = 'Epoch_average'
    average_df.set_index(['Epoch', 'Layer', 'Neuron'], inplace=True)

    # Reset the index of the original dataframe to prepare for merging
    df_reset = df.reset_index()

    # Combine the original data and the average data
    final_df = pd.concat([df_reset, average_df.reset_index()])

    # Set the index back to ["Epoch", "Layer", "Neuron"]
    final_df.set_index(['Epoch', 'Layer', 'Neuron'], inplace=True)

    return final_df

### t-Distributed Stochastic Neighbor Embedding (t-SNE)
t-SNE is a machine learning algorithm used for dimensionality reduction. It plots high dimensional objects on a lower-dimensional map and clusters them based by similarity using distance on the plot as a measure of difference. 

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

# Ensure all column names are strings
neuron_dfs.columns = neuron_dfs.columns.astype(str)

X = neuron_dfs
neuron_ids = neuron_dfs.index
X_tsne = TSNE(n_components=3, random_state=42).fit_transform(X)

plt.figure(figsize=(14, 10))
plt.scatter(X_tsne[:, 0], X_tsne[:, 1])

texts = []
for i, neuron_id in enumerate(neuron_ids):
    x_offset = 0.5 * (1 if np.random.rand() > 0.5 else -1)
    y_offset = 0.5 * (1 if np.random.rand() > 0.5 else -1)
    text = plt.text(X_tsne[i, 0] + x_offset, X_tsne[i, 1] + y_offset, str(neuron_id), fontsize=8)
    texts.append(text)

# Automatically adjust the text annotations to avoid overlaps
adjust_text(texts, only_move={'points':'y', 'text':'xy'}, arrowprops=dict(arrowstyle="-", color='gray', lw=0.5))

plt.title('t-SNE visualization of neuron activations')
plt.show()

### Interpretation: 
While the above plot doesn't give us much insight, it does show there is some underlying structural grouping of neurons which may provide the skeleton for a deeper analysis. 

In [ ]:
# Train/test/split the data

# Run model.trace_inference() over using X_test over num_epoch trials (100)

# Separate data sets when prediction is correct versus incorrect

# Compare distances between correct and incorrect



## Pairwise association of neural connections (WIP)

"Neurons the fire together, wire together"
This famous rule of thumb belongs to the realm of Hebbian learning in neurobiology. Does it also apply in the artifical context? 

One way about this is to try to find the neurons that are most closely associated and co-activated.

Prior to constructing the visualizations we should reorganize the weights so that we have a more maneuverable interface for tracing the connections between neurons.

As mere numbers, the weights aren't very human readable or traversable. But each weight contains a pair of information, that can be represented as a 2-tuple containing the following information
``(neuron_connected_to_current_neuron, strength_of_connection)``
In order to finally extract the neurons that are the most closely associated, we need to first make these connections more manifest. So we update our weights to be have these tuples

In [ ]:
print(model_df.columns)
def map_weights_input(df):
    # Create a dictionary to map the previous layer neuron IDs
    # TO DO: IMPORT THESE SETTINGS FROM A SINGLE SOURCE OF TRUTH TO REMOVE HARD CODING 
    previous_layer_mapping = {
        "L_input": [f"input_{i}" for i in range(20)],  
        "L_hidden_1": [f"n_{i}" for i in range(10)],  
        "L_output": [f"n_{i+10}" for i in range(10)],
    }

    def wrap_with_input_neuron(row, layer):
        previous_neurons = previous_layer_mapping[layer]
        # For weights that exceed the number of neurons in the previous layer, set the index part to None
        return [(previous_neurons[i] if i < len(previous_neurons) else None, val)
                for i, val in enumerate(row)]

    decorated_data = []
    for (epoch, layer, neuron), row in df.iterrows():
        # Check for columns to exclude from the mapping
        if neuron == any(["Final_Classification_Result"]):
            decorated_data.append(row.tolist())  # Keep the original data
        else:
            wrapped_row = wrap_with_input_neuron(row, layer)
            decorated_data.append(wrapped_row)

    # Reconstruct DataFrame with decorated values
    decorated_df = pd.DataFrame(decorated_data, index=df.index, columns=df.columns)

    return decorated_df

# Assuming the output of convert_nn_schema_to_df is stored in `model_df`
decorated_df = map_weights_input(model_df)
display(decorated_df)